In [1]:
import glob, sys

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, O, M, WikidataSemanticModelHelper
sys.path.append(str(ROOT_DIR / "sm_annotator"))

In [2]:
cwd = ROOT_DIR / "examples/semtab2020_novartis"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

In [3]:
gt = [
    ([O.SemanticModel.from_json(sm) for sm in r['semantic_models']], I.W2WTable.from_json(r['table']))
    for r in [M.deserialize_json(infile) for infile in glob.glob(str(cwd / "tables/*.json"))]
]

In [4]:
annotations = [grams.annotate(table) for sms, table in tqdm(gt)]

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
from sm_annotator.prelude import Annotator, SMVisualizer, widgets, GRAMSAnnotatorAssistant, SliderApp

qnodes = grams.qnodes.cache_dict()
wdprops = grams.wdprops
wdclasses = grams.wdclasses.cache_dict()

assistant = GRAMSAnnotatorAssistant([
    dict(table=table, sg=annotation.sg, dg=annotation.dg)
    for annotation, (sms, table) in zip(annotations, gt)
], qnodes, wdprops)

sm_visualizer_inputs_slider = [
    dict(description='', args=(table.id, '', WikidataSemanticModelHelper.minify_sm(sms[0]), WikidataSemanticModelHelper.minify_sm(annotation.sm), table, True)) 
    for annotation, (sms, table) in zip(annotations, gt)
]
sm_visualizer = SMVisualizer(qnodes, wdclasses, wdprops, cwd / "ground-truth", 
                             eshost='', username='', password='', dev=False, assistant=assistant)
slider = SliderApp(sm_visualizer, sm_visualizer.visualize)
slider.render(same_tab=True)

Output()

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='97243f34-8de4-41f4-9598-edf04b5b9e68')

HTML(value='<div id="01386d95-0598-4c01-a338-5d6d9a9fd93b"></div>')

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='d5a9f1b2-0353-4a34-bc06-2bf5c512d893')

<IPython.core.display.Javascript object>

In [15]:
slider.set_data(sm_visualizer_inputs_slider)